In [15]:
import torch

In [16]:
P = 100
N = 8
H = N + 1
M = 1

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
x = torch.randn(P, N).sign()
z = torch.prod(x, dim=1).view(P, M)
x, z = x.to(device), z.to(device) # Muevo los datos a gpu.

In [19]:
class mlp( torch.nn.Module): # Module para hacer modelos propios.
    def __init__(_, isize, hsize, osize): # Uso _ en lugar de self.
        super().__init__()  
        _.l1 = torch.nn.Linear(isize, hsize) # Params de modelo.
        _.l2 = torch.nn.Linear(hsize, osize)
    def forward( _, x):
        h = torch.tanh(_.l1( x)) # Grafo de computo.
        y = torch.tanh(_.l2( h))
        return y

In [20]:
model = mlp(N, H, M).to(device) # Muevo los parametros a gpu.
optim = torch.optim.SGD(model.parameters(), lr=0.1)
costf = torch.nn.MSELoss()

In [21]:
t, E = 0, 1.
model.train()
while E >= 0.01 and t < 9999:
    y = model(x)
    optim.zero_grad() # Optim sabe que resetear.
    error = costf(y, z)
    error.backward()
    optim.step() # Optim sabe que resetear.
    E = error.item()
    t += 1
    if t % 100 == 0:
        print(t, E)

100 0.8756523132324219
200 0.7571622729301453
300 0.6045585870742798
400 0.45880967378616333
500 0.3178875744342804
600 0.2095615267753601
700 0.1374921053647995
800 0.09086084365844727
900 0.06046449765563011
1000 0.04154100641608238
1100 0.030050022527575493
1200 0.022758018225431442
1300 0.017888229340314865
1400 0.014503926038742065
1500 0.012071696110069752
1600 0.01026791799813509


In [22]:
model.eval()
with torch.no_grad():
    y = model(x)
    E = costf(y, z).item()
    print(E)

0.009978143498301506
